In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [14]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [15]:
neighborhoods_data = newyork_data['features']

In [16]:
neighborhoods_data

[{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
   'type': 'Point'},
  'geometry_name': 'geom',
  'id': 'nyu_2451_34572.1',
  'properties': {'annoangle': 0.0,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661],
   'borough': 'Bronx',
   'name': 'Wakefield',
   'stacked': 1},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
   'type': 'Point'},
  'geometry_name': 'geom',
  'id': 'nyu_2451_34572.2',
  'properties': {'annoangle': 0.0,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012],
   'borough': 'Bronx',
   'name': 'Co-op City',
   'stacked': 2},
  'type': 'Feature'},
 {'geometry': {'coordinates': [-73.82780644716412, 40.887555677350775],
   'type': 'Point'},
  'geometry_name': 'geom',


In [17]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

In [10]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


In [24]:
address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

c:\program files\python36\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [26]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_newyork)  
    
map_newyork

In [20]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [28]:
manhattan_data.loc[0, 'Neighborhood']
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


In [31]:
CLIENT_ID = 'NAIWMQ5DRTM1QQRNAT51LM4HI0FCC44AXHKYOTXBC5XQYBBO'
CLIENT_SECRET = 'CQSF1ZUFUIH2QVFKJ54OXTKL1WQQIDADXBNSHWCUHIV4AAK0'
VERSION = '20180606'
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [34]:
results = requests.get(url).json()

In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

nearby_venues

,reasons.count,reasons.items,referralId,venue.categories,venue.delivery.id,venue.delivery.provider.icon.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.name,venue.delivery.url,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b4429abf964a52037f225e3-0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",72548,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/arturos-pizza-51...,4b4429abf964a52037f225e3,5198 Broadway,US,New York,United States,at 225th St.,240,"[5198 Broadway (at 225th St.), New York, NY 10...","[{'label': 'display', 'lat': 40.87441177110231...",40.874412,-73.910271,NaN,10463,NY,Arturo's,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4baf59e8f964a520a6f93be3-1,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",NaN,NaN,NaN,NaN,NaN,NaN,4baf59e8f964a520a6f93be3,5500 Broadway,US,Bronx,United States,230th Street,376,"[5500 Broadway (230th Street), Bronx, NY 10463...","[{'label': 'display', 'lat': 40.87684369079793...",40.876844,-73.906204,NaN,10463,NY,Bikram Yoga,0,[],NaN
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b79cc46f964a520c5122fe3-2,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,4b79cc46f964a520c5122fe3,3033 Tibbett Ave,US,Bronx,United States,btwn 230th & 231st,452,"[3033 Tibbett Ave (btwn 230th & 231st), Bronx,...","[{'label': 'display', 'lat': 40.8804044222466,...",40.880404,-73.908937,NaN,10463,NY,Tibbett Diner,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-55f81cd2498ee903149fcc64-3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,55f81cd2498ee903149fcc64,171 W 230th St,US,Bronx,United States,Kimberly Pl,441,"[171 W 230th St (Kimberly Pl), Bronx, NY 10463...","[{'label': 'display', 'lat': 40.87753134921497...",40.877531,-73.905582,NaN,10463,NY,Starbucks,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b5357adf964a520319827e3-4,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,4b5357adf964a520319827e3,5501 Broadway,US,Bronx,United States,W 230th St,353,"[5501 Broadway (W 230th St), Bronx, NY 10463, ...","[{'label': 'display', 'lat': 40.87699277991182...",40.876993,-73.906507,NaN,10463,NY,Dunkin' Donuts,0,[],NaN
5,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b9c9c6af964a520b27236e3-5,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",277380,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/land--sea-restau...,4b9c9c6af964a520b27236e3,5535 Broadway,US,Bronx,United States,231st St,429,"[5535 Broadway (231st St), Bronx, NY 10463, Un...","[{'label': 'display', 'lat': 40.87788463309788...",40.877885,-73.905873,NaN,10463,NY,Land & Sea Restaurant,0,[],NaN
6,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-546d31ca498e561c698a0320-6,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",NaN,NaN,NaN,NaN,NaN,NaN,546d31ca498e561c698a0320,NaN,US,Bronx,United States,NaN,478,"[Bronx, NY, United States]","[{'label': 'display', 'lat': 40.87723198343352...",40.877232,-73.905042,NaN,NaN,NY,T.J. Maxx,0,[],NaN
7,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-57655be738faa66160da7527-7,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,57655be738faa66160da7527,50 W 225th St,US,New York,United States,NaN,355,"[50 W 225th St, New York, NY 10463, United Sta...","[{'label': 'display', 'lat': 40.87375455421851...

In [37]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Arturo's,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",40.874412,-73.910271
1,Bikram Yoga,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",40.876844,-73.906204
2,Tibbett Diner,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",40.880404,-73.908937
3,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",40.877531,-73.905582
4,Dunkin' Donuts,"[{'id': '4bf58dd8d48988d148941735', 'name': 'D...",40.876993,-73.906507
5,Land & Sea Restaurant,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",40.877885,-73.905873
6,T.J. Maxx,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",40.877232,-73.905042
7,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",40.873755,-73.908613
8,TCR The Club of Riverdale,"[{'id': '4e39a891bd410d7aed40cbc2', 'name': 'T...",40.878628,-73.914568
9,Blink Fitness Riverdale,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",40.877147,-73.905837


In [38]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin' Donuts,Donut Shop,40.876993,-73.906507
5,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873
6,T.J. Maxx,Department Store,40.877232,-73.905042
7,Starbucks,Coffee Shop,40.873755,-73.908613
8,TCR The Club of Riverdale,Tennis Stadium,40.878628,-73.914568
9,Blink Fitness Riverdale,Gym,40.877147,-73.905837


In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [41]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3297, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop


# analyze each neighborhood

In [42]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spi

In [43]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Spi

In [44]:
manhattan_grouped.shape

(40, 323)

In [45]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.08
1                Park  0.07
2               Hotel  0.05
3           Wine Shop  0.03
4  Italian Restaurant  0.03


----Carnegie Hill----
                 venue  freq
0          Pizza Place  0.06
1       Cosmetics Shop  0.05
2          Coffee Shop  0.05
3                 Café  0.04
4  Japanese Restaurant  0.03


----Central Harlem----
                venue  freq
0  African Restaurant  0.07
1  Chinese Restaurant  0.05
2   French Restaurant  0.05
3      Cosmetics Shop  0.05
4  Seafood Restaurant  0.05


----Chelsea----
                venue  freq
0  Italian Restaurant  0.06
1         Coffee Shop  0.06
2      Ice Cream Shop  0.04
3           Nightclub  0.04
4              Bakery  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1    American Restaurant  0.04
2     Dim Sum Restaurant  0.04
3  Vietnamese Restaurant  0.04
4      Hotpot Restaurant  0.03


----Civic Center----


In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Gym,Italian Restaurant,Boat or Ferry,Burger Joint,Bakery,BBQ Joint
1,Carnegie Hill,Pizza Place,Cosmetics Shop,Coffee Shop,Café,Japanese Restaurant,Spa,French Restaurant,Gym,Bar,Yoga Studio
2,Central Harlem,African Restaurant,Seafood Restaurant,French Restaurant,American Restaurant,Cosmetics Shop,Chinese Restaurant,Event Space,Beer Bar,Caribbean Restaurant,Gym / Fitness Center
3,Chelsea,Italian Restaurant,Coffee Shop,Nightclub,Ice Cream Shop,Bakery,American Restaurant,Hotel,Theater,Seafood Restaurant,Art Gallery
4,Chinatown,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Cocktail Bar,Bakery,Hotpot Restaurant,Ice Cream Shop,Noodle House
5,Civic Center,Gym / Fitness Center,Bakery,Cocktail Bar,Yoga Studio,French Restaurant,Spa,Sushi Restaurant,Sandwich Place,Coffee Shop,Park
6,Clinton,Theater,Italian Restaurant,American Restaurant,Coffee Shop,Hotel,Spa,Wine Shop,Gym / Fitness Center,Gym,Indie Theater
7,East Harlem,Mexican Restaurant,Bakery,Latin American Restaurant,Pizza Place,Deli / Bodega,Spa,Thai Restaurant,Convenience Store,Café,Taco Place
8,East Village,Ice Cream Shop,Bar,Wine Bar,Mexican Restaurant,Cocktail Bar,Italian Restaurant,Pizza Place,Chinese Restaurant,Vegetarian / Vegan Restaurant,Speakeasy
9,Financial District,Coffee Shop,Hotel,Gym,Steakhouse,Food Truck,Japanese Restaurant,Wine Shop,Italian Restaurant,Bar,Pizza Place


In [48]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 0, 0, 0, 0, 3, 4, 0, 3])

In [49]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Discount Store,Coffee Shop,Yoga Studio,Diner,Steakhouse,Supplement Shop,Shoe Store,Tennis Stadium,Bank,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Dim Sum Restaurant,American Restaurant,Vietnamese Restaurant,Bubble Tea Shop,Cocktail Bar,Bakery,Hotpot Restaurant,Ice Cream Shop,Noodle House
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Pizza Place,Bakery,Sandwich Place,Grocery Store,Supermarket,Deli / Bodega,Chinese Restaurant,Mobile Phone Shop,Lounge
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Pizza Place,Café,Wine Bar,Bakery,American Restaurant,Restaurant,Pharmacy,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Bakery,Sandwich Place,Cocktail Bar,School,Chinese Restaurant


In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# examine each cluster

In [51]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Pizza Place,Bakery,Sandwich Place,Grocery Store,Supermarket,Deli / Bodega,Chinese Restaurant,Mobile Phone Shop,Lounge
3,Inwood,Mexican Restaurant,Lounge,Pizza Place,Café,Wine Bar,Bakery,American Restaurant,Restaurant,Pharmacy,Frozen Yogurt Shop
4,Hamilton Heights,Mexican Restaurant,Café,Coffee Shop,Pizza Place,Yoga Studio,Bakery,Sandwich Place,Cocktail Bar,School,Chinese Restaurant
5,Manhattanville,Seafood Restaurant,Deli / Bodega,Mexican Restaurant,Italian Restaurant,Chinese Restaurant,Bike Trail,Sushi Restaurant,Supermarket,Burger Joint,Bus Station
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Coffee Shop,Juice Bar,Bakery,French Restaurant,Spa,Hotel,Boutique
10,Lenox Hill,Coffee Shop,Italian Restaurant,Sushi Restaurant,Gym / Fitness Center,Pizza Place,Deli / Bodega,Gym,Burger Joint,Salad Place,Cocktail Bar
11,Roosevelt Island,Sandwich Place,Park,Indie Theater,Bus Station,Supermarket,Farmers Market,Outdoors & Recreation,Metro Station,Greek Restaurant,Gym
12,Upper West Side,Italian Restaurant,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Indian Restaurant,Wine Bar,Bakery,Cosmetics Shop,Ice Cream Shop,Mediterranean Restaurant
16,Murray Hill,Hotel,Burger Joint,Italian Restaurant,Coffee Shop,Bar,Gym,Spa,Japanese Restaurant,American Restaurant,French Restaurant
17,Chelsea,Italian Restaurant,Coffee Shop,Nightclub,Ice Cream Shop,Bakery,American Restaurant,Hotel,Theater,Seafood Restaurant,Art Gallery


In [52]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Little Italy,Bakery,Chinese Restaurant,Café,Yoga Studio,Pizza Place,Cocktail Bar,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bubble Tea Shop
